# Scopone Scientifico
This notebook will go though the creation of a "Python library" that allows to simulate game of Scopone Scientifico.

In [ ]:
%pip install tqdm
%pip install numpy
%pip install pandas

In [ ]:
import random
from typing import List, Callable
import itertools
from tqdm import tqdm

In [ ]:
class Card:
    def __init__(self, rank: int, suit: str):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        rank_raster = self.rank

        if rank_raster == 10:
            rank_raster = "King"
        elif rank_raster == 9:
            rank_raster = "Queen"
        elif rank_raster == 8:
            rank_raster = "Jack"

        if self.suit == "bello":
            return f"{self.rank} {self.suit}"
        else:
            return f"{self.rank} di {self.suit}"

In [ ]:
class Deck:
    suits = ['picche', 'bello', 'fiori', 'cuori']
    ranks = list(range(1, 11))  # Ranks from 1 to 7, plus 8, 9, and 10 for face cards.

    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards, )

    def deal(self, num_cards: int) -> List[Card]:
        return [self.cards.pop() for _ in range(num_cards)]
    
    def __str__(self):
        result = '#' * 10 + f' Deck {self.__hash__()} ' + '#' * 10 + '\n'
        for card in self.cards:
            result += str(card) + '\n'
        result += '#' * 20 + '\n'
        result += f'{len(self.cards)} cards in the deck.\n'
        for suit in self.suits:
            result += f'{sum(1 for card in self.cards if card.suit == suit)} {suit}\n'
        result += '#' * 20
        return result

    def reset(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        self.shuffle()



In [ ]:
deck = Deck()

print(deck)

In [ ]:
class Player:
    def __init__(self, side: int):
        if side not in [1, 2]:
            raise ValueError("Side must be 1 or 2.")
        self.side = side
        self.hand = []
        self.captures = []
        self.scopas = 0


    def play_card(self, card_index: int, v= 0) -> Card:
        card = self.hand.pop(card_index)
        if v >= 2: print(f'[PLAYER] Player {self.__hash__()} played {card}.')
        return card

    def capture(self, cards: List[Card], _with: Card):
        for card in cards:
            self.captures.append(card)
        if _with is not None: self.hand.remove(_with)

    def scopa(self):
        self.scopas += 1
    

    def __str__(self):
        return f'[PLAYER] Player {self.__hash__()} for side {self.side} has {len(self.hand)} cards in hand and {len(self.captures)} captures.'
    
    def show_hand(self):
        out = '#' * 10 + f' Player {self.__hash__()} ' + '#' * 10 + '\n'
        for card in self.hand:
            out += str(card) + '\n'
        out += '#' * 20
        return out
    
    def reset(self):
        self.hand = []
        self.captures = []
        self.scopas = 0

In [ ]:
class ScoponeGame:
    def __init__(self):
        self.deck = Deck()
        self.players = [Player(i) for i in [1,2,1,2]]
        self.table = []
        self.last_capture = None

    def deal_initial_hands(self):
        self.deck.reset()
        for player in self.players:
            player.reset()
            player.hand = self.deck.deal(10)

    def __str__(self):
        return f"Players: {[player.__hash__() for player in self.players]}, Table: {self.table}"
    
    def player_details(self):
        return [str(player) for player in self.players]
    
    def card_in_table(self, card):

        if len(self.table) == 0:
            return False, []
        

        current_table = [self.table[i] for i in range(len(self.table))]
        all_combinations = []
        for i in range(1, len(current_table) + 1):
            all_combinations.extend(list(itertools.combinations(current_table, i)))

        for comb in all_combinations:
            for c in comb:
                if c is List:
                    if sum([cc.rank for cc in c]) == card.rank:
                        return True, c
                    
            if c.rank == card.rank:
                return True, c
        return False, []

    def describe_status(self) -> str:
        out = '#' * 10 + ' Game Status ' + '#' * 10 + '\n'
        out += 'Table:\n'
        for card in self.table:
            out += str(card) + '\n'
        out += '#' * 20 + '\n'
        for player in self.players:
            out += 'Player ' + str(player.__hash__()) + f' for side {player.side}\n'
            out += 'Hand:\n'
            out += player.show_hand() + '\n'
            out += 'Captured stack:\n'
            out += f'{[str(c) for c in player.captures]}\n'
        out += '#' * 20 + '\n'

        return out
        
    
    def play_card(self, card, player, v=0):
        if v >= 2: print(f'[GAME] Player {player.__hash__()} plays {card}')

        # ACE CASE
        if card.rank == 1:
                self.table.append(card)
                player.capture(self.table, _with=card)
                if v >= 2: 
                    print(f'[GAME] Player {player.__hash__()} captures {[str(c) for c in self.table]} with {card}')
                self.table=[]
                return


        isin, comb = self.card_in_table(card=card)

        if isin:
            self.last_capture = player
            # TODO comb is important as the agent will have to be able to chose the best capture
            comb = [comb]

            for c in comb:
                self.table.remove(c)

            comb.append(card)
            player.capture(comb, _with=card)

            if self.table == []:
                player.scopa()
                if v >= 2: print(f'[GAME] Player {player.__hash__()} scopa!')
            
            if v >= 2: 
                print(f'[GAME] Player {player.__hash__()} captures {[str(c) for c in comb]} with {card}')
        else:
            player.play_card(player.hand.index(card), v=v)
            self.table.append(card)

    def evaluate_round(self, players: List[Player], v=0) -> List[int]:

        side1_points = 0
        side2_points = 0


        side1 = []
        side2 = []
        for player in players:
            for card in player.captures:
                if player.side == 1:
                    side1.append(card)
                elif player.side == 2: 
                    side2.append(card)
            if player.scopas > 0:
                if player.side == 1:
                    side1_points += player.scopas
                elif player.side == 2:
                    side2_points += player.scopas

        if len(side1) + len(side2) != 40:
            raise ValueError("Not all cards have been captured.")

        # Key evaulation

        if v >= 2: print(f'[EVAL] Side 1: {side1_points} Side 2: {side2_points}. Next up: Sette Bello')
        
        #SetteBello
        for card in side1:
            if card.rank == 7 and card.suit == 'bello':
                side1_points += 1
                break
        if side1_points == 0:
            side2_points += 1


        if v >= 2: print(f'[EVAL] Side 1: {side1_points} Side 2: {side2_points}. Next up: Cards')
        #Cards
        # Only possible tie is 20 cards each and in that case no points are awarded
        if len(side1) > len(side2):
            side1_points += 1
        elif len(side1) < len(side2):
            side2_points += 1


        if v >= 2: print(f'[EVAL] Side 1: {side1_points} Side 2: {side2_points}. Next up: Ori')
        #Ori
        counter = 0
        for card in side1:
            if card.suit == 'bello':
                counter += 1
        if counter > 5:
            side1_points += 1
        elif counter < 5:
            side2_points += 1

        if v >= 2: print(f'[EVAL] Side 1: {side1_points} Side 2: {side2_points}. Next up: Primiera')
        #primiera
        score1 = [0,0,0,0]
        score2= [0,0,0,0]
        for i,suit in enumerate(['bello', 'picche', 'fiori', 'cuori']):
            for card in side1:
                if card.suit == suit and card.rank >= score1[i]:
                    score1[i] = card.rank
            for card in side2:
                if card.suit == suit and card.rank >= score2[i]:
                    score2[i] = card.rank
        if sum(score1) > sum(score2):
            side1_points += 1
        elif sum(score1) < sum(score2):
            side2_points += 1

        if v >= 2: print(f'[EVAL] Side 1: {side1_points} Side 2: {side2_points}. Next up: Napola')

        #Napola

        side1_belli = []
        side2_belli = []

        for card in side1:
            if card.suit == 'bello':
                side1_belli.append(card)
        for card in side2: 
            if card.suit == 'bello':
                side2_belli.append(card)

        
        side1_belli_ranks = sorted([card.rank for card in side1_belli])
        side2_belli_ranks = sorted([card.rank for card in side2_belli])

        def calculate_sequence_points(ranks):
            points = 0
            if all(rank in ranks for rank in [1, 2, 3]):
                points = 3
                for rank in range(4, 11):
                    if rank in ranks:
                        points += 1 
                    else:
                        break
            return points

        side1_points += calculate_sequence_points(side1_belli_ranks)
        side2_points += calculate_sequence_points(side2_belli_ranks)

        if v >= 2: print(f'[EVAL] Final Score - Side 1: {side1_points} Side 2: {side2_points}')

        return [side1_points, side2_points]
    
    
    
    def play_game(self, v = 0):
        i=0
        while [len(player.hand) == 0 for player in self.players] != [True, True, True, True]:
            if v >= 2: print('#' * 20 + f' Turn {i+1} ' + '#' * 20)
            for player in self.players:
                self.play_card(player.hand[random.randint(0, len(player.hand) - 1)], player, v=v)
            if v >= 2: print('#'*48)

            if [len(player.hand) == 0 for player in self.players] == [True, True, True, True]:
                self.last_capture.capture(self.table, _with=None)
                if v >= 2: print(f'[GAME] Player {self.last_capture.__hash__()} captures the table.')
                if v >= 2: print(f'[GAME] {[str(c) for c in self.table]}')
                self.table = []
                if v >= 2: print('[GAME] \n\n\n>>>>>>>>>>>>>>>>Game over!\n\n\n')
                return
            i+=1

    def is_match_over(self, side1_score, side2_score, winning_threshold = 21, v = 0):

        if abs(side1_score-side2_score) == 1 and min(side1_score, side2_score) >= winning_threshold-1:
            if v >= 1: print(f'[MATCH] DEUCE! old threshold: {winning_threshold} new threshold: {max(side1_score, side2_score) + 1}')
            winning_threshold = max(side1_score, side2_score) + 1

        if side1_score >= winning_threshold and side2_score < side1_score - 1:
            return True
        elif side2_score >= winning_threshold and side1_score < side2_score - 1:
            return True

        return False

    def play_match(self, v = 0, winning_threshold = 21):
        if v == 1: print(f'[MATCH] Starting match with winning threshold {winning_threshold}')
        side1_score = []
        side2_score = []
        i = 0
        while not self.is_match_over(sum(side1_score), sum(side2_score), winning_threshold, v=v):
            self.deal_initial_hands()
            self.play_game(v=v)
            if v>= 2: print(self.describe_status())
            scores = self.evaluate_round(self.players, v=v)
            side1_score.append(scores[0])
            side2_score.append(scores[1])
            if v >= 1: print(f'[MATCH] ROUND {i+1} \t|\tSide 1:\t{sum(side1_score)} Side 2:\t{sum(side2_score)}')
            i+=1

            if i > 50:
                raise ValueError("Too many rounds played.")
        if v >= 1: print(f'[MATCH] --------\n[MATCH] RESULTS \t|\tSide 1:\t{sum(side1_score)} Side 2:\t{sum(side2_score)} ')

        
        if sum(side1_score) > sum(side2_score):
            return 1
        else:
            return 2


In [ ]:
game = ScoponeGame()


import time 
start = time.time()
print(game.play_match(v=1))
end = time.time()

print(f'Time elapsed: {end-start} seconds')